In [3]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle (a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    rad = np.abs(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1],a[0]-b[0]))
    angle=(np.abs(rad* 180.0/np.pi))
    #return angle
    return angle

In [4]:
# set_Matrix
#set_D
D=np.zeros((4,4))
D[0][2]=1
D[1][3]=1
#D[4][6]=1
#D[5][7]=1
#print("D = ")
#print(D)

# D ^n:
def matrixPow(Matrix,n):
    if(type(Matrix)==list):
        Matrix=np.array(Matrix)
    if(n==1):
        return Matrix
    else:
        return np.matmul(Matrix,matrixPow(Matrix,n-1))
#F
def get_F(D,n,t):
    F=0
    for i in range(1, n+1):
        F=F+(matrixPow((D*t),i))/math.factorial(i)
    F=F+np.eye(D.shape[0])
    print("F = ")
    print(F)
    return F

# H 
def get_H(num_measurement,num_state):
    H= np.zeros((num_measurement,num_state))
    print(H)
    for i in range(num_state):
        for j in range(num_measurement):
            if(i==j):
                H[i][j]=1
                break
    return H

# Q :
def get_Q(D,F):
    r=np.zeros((F.shape[1],1))
   # print(F[[F.shape[1]-4],[F.shape[1]-1]])
    #print(F)
   # print("r.shape",r.shape)
    copyline=D.shape[0]
   # print("copyline:",copyline)
    i=0
    j=0
    
    while(i<r.shape[0]):
        if j < copyline:
         #   print("i",i,"j",j)
       #     print(F[[F.shape[1]-4+j],[F.shape[1]-1]])
            r[[i],[0]]=F[[F.shape[1]-4+j],[F.shape[1]-1]]
            j=j+1
            i=i+1
        else:
            j=0

  #      np.array(r)[0][i]=np.array(F)[i][F.shape[1]-1]
    print("r = ")
    print(r)
 #   print("r_T = ")
#    print(r.T)
    Q = r*r.T
#    print("Q =")
#    print (Q)
    
    return Q



In [5]:
# 合并

# 33*4=132
# Z= np.zeros((132,132))
def new_D(D,n): #n number of point
 #   print("D:")
 #   print(D)
    Z= np.zeros((D.shape[0]*n,D.shape[0]*n))
#    print("new D is ",D.shape[0]*n,"*",D.shape[0]*n,"Matrix")
    First_one=0
    Second_one=0
    i = 0
    while(i < Z.shape[0]):  
        
        for j in range(Z.shape[1]):
            if i==j:
                if First_one==0:
                    b=j+2
                    Z[i][b]=1
                    First_one=1
                    
                ##    print("i=",i)
                  #  print("j=",b)
                   # print("第一个 1：")
                    #print(Z)
                    Second_one==0
                    i=i+1
                    break
    
            if First_one==1:
                b=b+1
                Z[i][b]=1
                i=i+3
                First_one=0
                
               ## print("i=",i)
               # print("j=",b)
                #print("第二个 1：")
                #print(Z)
                
                if i>=Z.shape[0]-1 & b>=Z.shape[1]-1:
                    print("new D: ")
                    print(Z)
                    return Z
                b=0
                break
#newD=new_D(D,2)
#F=get_F(newD,1,1)# (D,n,t)
#Q=get_Q(D,F)

In [11]:
## Kalman
def set_kalman():
    kalman = cv2.KalmanFilter(8,4) #（x，y，dx，dy) # x1,y1,dx1,dy1,x2,y3,dx2,dy2
# measurementMatrix H 
    #kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]],np.float32)
    #get_H(num_measurement,num_state):
    kalman.measurementMatrix = np.array(get_H(4,8),np.float32)
# transitionMatrix F 
    #kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]],np.float32)
    
    #get_F new_D(D,n): #n number of point
    F=get_F(new_D(D,2),1,1) # (D,n,t)

    kalman.transitionMatrix = np.array(F,np.float32)
# processNoiseCov Q
    #kalman.processNoiseCov = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],np.float32) * 0.03
    Q=get_Q(D,F)
    kalman.processNoiseCov = np.array(Q,np.float32) * 0.03
    
    print("kalman.measurementMatrix:",kalman.measurementMatrix)
    print("kalman.transitionMatrix: ",kalman.transitionMatrix)
    print("kalman.processNoiseCov:",Q)
    return kalman

In [14]:
def video():
    cap =cv2.VideoCapture(0)
    kalman = set_kalman()
# Curl counter variables
    counter=0
    stage = None
    last_measurement = current_measurement = np.array((4,1),np.float32)
    last_predicition = current_prediction = np.zeros((4,1),np.float32)
    
    # Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            #global current_measurement,measurements,last_measurement,current_prediction,last_prediction
            
            ret,frame = cap.read()
        
        # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
        #Make detection
            results = pose.process(image)
        
        # Recolor image to BGR
            image.flags.writeable =True
            image= cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # print (results)
        # cv2.imshow("Mediapipe Feed", frame)
        
        # Extrack landmarks
            
        
            if results.pose_landmarks==None:
                continue
            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
            #print (landmarks)
            last_measurement = current_measurement
            last_prediction = current_prediction
            measurent= [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
      
            current_measurement = np.array(np.float32(measurent))
                
            kalman.correct(current_measurement)
            print ('measurement: ',current_measurement)
            current_prediction = kalman.predict()
            print("prediction: ", current_prediction)
            
            #new_shoulder = [[float(current_prediction[0]),float(current_prediction[1])]]
            #print('new: ',new_shoulder)
            
            # Calculate angle
            angle=calculate_angle(shoulder,elbow,wrist)
                
                #current_measurement = np.array([angle])
              #  kalman.correct(current_measurement)
              #  current_prediction = kalman.predict()
                
            # Curl counter logic
            if angle > 160:
                stage ="down"
            if angle < 45 and stage == 'down':
                stage ="up"
                counter +=1
               # print (counter)
            
     # Visualize angle
            
            #position=tuple(np.multiply(elbow, [640,480]).astype(int))
            position=(100, 100)
            font =cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(image, str(counter),position,font,2,(255,255,255),2,cv2.LINE_AA)
                                      
            #print (landmarks)
            
        
        # Render detections
            mp_drawing.draw_landmarks (image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  )
            cv2.imshow("Mediapipe Feed", image)

            if cv2.waitKey(10) & 0xFF == ord ('q'):
                break
        
    cap.release()
    cv2.destroyAllWindows() 
    cv2.waitKey(1)                       
             
            

In [15]:
video()

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
new D: 
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
F = 
[[1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
r = 
[[0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]]
kalman.measurementMatrix: [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]
kalman.transitionMatrix:  [[1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
kalman.proce

alt:  [0.9188713 0.5974292 0.9560337 0.8869801]
new shoulder:  [[0.        ]
 [1.0632287 ]
 [0.        ]
 [0.10047975]
 [0.        ]
 [1.0632287 ]
 [0.        ]
 [0.10047975]]
alt:  [0.9188722  0.59697896 0.9689374  0.8845448 ]
new shoulder:  [[0.        ]
 [1.0611167 ]
 [0.        ]
 [0.09986468]
 [0.        ]
 [1.0611167 ]
 [0.        ]
 [0.09986468]]
alt:  [0.9189509  0.59696686 0.971741   0.88390076]
new shoulder:  [[0.       ]
 [1.0594794]
 [0.       ]
 [0.0994847]
 [0.       ]
 [1.0594794]
 [0.       ]
 [0.0994847]]
alt:  [0.91859853 0.59637374 0.97166216 0.8813975 ]
new shoulder:  [[0.        ]
 [1.0576259 ]
 [0.        ]
 [0.09908174]
 [0.        ]
 [1.0576259 ]
 [0.        ]
 [0.09908174]]
alt:  [0.91861564 0.5963121  0.9733909  0.8808074 ]
new shoulder:  [[0.        ]
 [1.0562701 ]
 [0.        ]
 [0.09886859]
 [0.        ]
 [1.0562701 ]
 [0.        ]
 [0.09886859]]
alt:  [0.91860694 0.596304   0.97098005 0.88024735]
new shoulder:  [[0.        ]
 [1.0553417 ]
 [0.        ]
 [0

alt:  [0.92189515 0.594355   0.9981761  0.89397955]
new shoulder:  [[0.        ]
 [1.0618159 ]
 [0.        ]
 [0.10124227]
 [0.        ]
 [1.0618159 ]
 [0.        ]
 [0.10124227]]
alt:  [0.921854   0.5942544  0.9953987  0.89443326]
new shoulder:  [[0.        ]
 [1.0614178 ]
 [0.        ]
 [0.10108449]
 [0.        ]
 [1.0614178 ]
 [0.        ]
 [0.10108449]]
alt:  [0.9215932  0.5937129  0.9949233  0.88782984]
new shoulder:  [[0.        ]
 [1.0596322 ]
 [0.        ]
 [0.10048298]
 [0.        ]
 [1.0596322 ]
 [0.        ]
 [0.10048298]]
alt:  [0.9215685  0.59363043 0.9892557  0.8867503 ]
new shoulder:  [[0.        ]
 [1.0580473 ]
 [0.        ]
 [0.10004198]
 [0.        ]
 [1.0580473 ]
 [0.        ]
 [0.10004198]]
alt:  [0.9215447  0.5922999  0.9886569  0.88535225]
new shoulder:  [[0.        ]
 [1.0559826 ]
 [0.        ]
 [0.09956621]
 [0.        ]
 [1.0559826 ]
 [0.        ]
 [0.09956621]]
alt:  [0.9216242 0.5922935 0.988891  0.8853523]
new shoulder:  [[0.       ]
 [1.0546005]
 [0.       

alt:  [0.93306416 0.6137072  0.974767   0.94174635]
new shoulder:  [[0.        ]
 [1.1039652 ]
 [0.        ]
 [0.10735997]
 [0.        ]
 [1.1039652 ]
 [0.        ]
 [0.10735997]]
alt:  [0.93196714 0.6131006  0.9715265  0.9444925 ]
new shoulder:  [[0.        ]
 [1.1054093 ]
 [0.        ]
 [0.10748679]
 [0.        ]
 [1.1054093 ]
 [0.        ]
 [0.10748679]]
alt:  [0.9312402  0.61308056 0.9815496  0.9430371 ]
new shoulder:  [[0.        ]
 [1.1059332 ]
 [0.        ]
 [0.10734341]
 [0.        ]
 [1.1059332 ]
 [0.        ]
 [0.10734341]]
alt:  [0.9036228  0.6125821  0.97966206 0.9194435 ]
new shoulder:  [[0.        ]
 [1.1015786 ]
 [0.        ]
 [0.10552917]
 [0.        ]
 [1.1015786 ]
 [0.        ]
 [0.10552917]]
alt:  [0.8722739  0.6068289  0.97907144 0.8282909 ]
new shoulder:  [[0.        ]
 [1.0786717 ]
 [0.        ]
 [0.09763123]
 [0.        ]
 [1.0786717 ]
 [0.        ]
 [0.09763123]]
alt:  [0.83503234 0.60139865 0.9785365  0.7988962 ]
new shoulder:  [[0.        ]
 [1.0531266 ]
 [0. 

alt:  [0.4862557  0.21940178 0.5788765  0.24040759]
new shoulder:  [[0.        ]
 [0.38082394]
 [0.        ]
 [0.02431018]
 [0.        ]
 [0.38082394]
 [0.        ]
 [0.02431018]]
alt:  [0.48584777 0.21137685 0.5799618  0.22528212]
new shoulder:  [[0.        ]
 [0.35280144]
 [0.        ]
 [0.01851638]
 [0.        ]
 [0.35280144]
 [0.        ]
 [0.01851638]]
alt:  [0.48630062 0.20832555 0.5856782  0.21505482]
new shoulder:  [[0.        ]
 [0.33121005]
 [0.        ]
 [0.01524685]
 [0.        ]
 [0.33121005]
 [0.        ]
 [0.01524685]]
alt:  [0.48553285 0.20791806 0.58240914 0.22738081]
new shoulder:  [[0.        ]
 [0.32018852]
 [0.        ]
 [0.01540266]
 [0.        ]
 [0.32018852]
 [0.        ]
 [0.01540266]]
alt:  [0.48515764 0.20869444 0.5735048  0.25085178]
new shoulder:  [[0.        ]
 [0.3196519 ]
 [0.        ]
 [0.01843653]
 [0.        ]
 [0.3196519 ]
 [0.        ]
 [0.01843653]]
alt:  [0.48542243 0.21390587 0.56200415 0.2892601 ]
new shoulder:  [[0.        ]
 [0.33148494]
 [0. 

...

alt:  [0.46728864 0.2247829  0.541401   0.36192733]
new shoulder:  [[0.        ]
 [0.40457496]
 [0.        ]
 [0.03917336]
 [0.        ]
 [0.40457496]
 [0.        ]
 [0.03917336]]
alt:  [0.46755832 0.2286832  0.5325158  0.37167487]
new shoulder:  [[0.        ]
 [0.41162482]
 [0.        ]
 [0.04137663]
 [0.        ]
 [0.41162482]
 [0.        ]
 [0.04137663]]
alt:  [0.46767506 0.23120384 0.5307645  0.37241286]
new shoulder:  [[0.        ]
 [0.41825396]
 [0.        ]
 [0.04297152]
 [0.        ]
 [0.41825396]
 [0.        ]
 [0.04297152]]
alt:  [0.46771336 0.23247282 0.5311391  0.37198868]
new shoulder:  [[0.        ]
 [0.42332155]
 [0.        ]
 [0.04382752]
 [0.        ]
 [0.42332155]
 [0.        ]
 [0.04382752]]
alt:  [0.46765786 0.23309337 0.53059185 0.37417924]
new shoulder:  [[0.        ]
 [0.4271657 ]
 [0.        ]
 [0.04427013]
 [0.        ]
 [0.4271657 ]
 [0.        ]
 [0.04427013]]
alt:  [0.46795374 0.23617247 0.53017986 0.37599745]
new shoulder:  [[0.        ]
 [0.43130004]
 [0. 

alt:  [0.47217888 0.24537805 0.51454604 0.3970588 ]
new shoulder:  [[0.        ]
 [0.46607083]
 [0.        ]
 [0.05150093]
 [0.        ]
 [0.46607083]
 [0.        ]
 [0.05150093]]
alt:  [0.47218475 0.24454378 0.5192894  0.39121425]
new shoulder:  [[0.        ]
 [0.46294743]
 [0.        ]
 [0.04903201]
 [0.        ]
 [0.46294743]
 [0.        ]
 [0.04903201]]
alt:  [0.47221464 0.24410242 0.5280625  0.38618988]
new shoulder:  [[0.        ]
 [0.45829555]
 [0.        ]
 [0.04669628]
 [0.        ]
 [0.45829555]
 [0.        ]
 [0.04669628]]
alt:  [0.47214478 0.24375199 0.52836615 0.3666074 ]
new shoulder:  [[0.        ]
 [0.4504601 ]
 [0.        ]
 [0.04370205]
 [0.        ]
 [0.4504601 ]
 [0.        ]
 [0.04370205]]
alt:  [0.47221804 0.2285072  0.52690065 0.35370275]
new shoulder:  [[0.        ]
 [0.4339915 ]
 [0.        ]
 [0.03921421]
 [0.        ]
 [0.4339915 ]
 [0.        ]
 [0.03921421]]
alt:  [0.47109094 0.22549805 0.5262306  0.35473195]
new shoulder:  [[0.       ]
 [0.4210411]
 [0.   

alt:  [0.48347384 0.23102133 0.5230987  0.37989926]
new shoulder:  [[0.        ]
 [0.4294824 ]
 [0.        ]
 [0.04298277]
 [0.        ]
 [0.4294824 ]
 [0.        ]
 [0.04298277]]
alt:  [0.4891441  0.23093109 0.5231739  0.37966487]
new shoulder:  [[0.        ]
 [0.42927667]
 [0.        ]
 [0.0429059 ]
 [0.        ]
 [0.42927667]
 [0.        ]
 [0.0429059 ]]
alt:  [0.49404725 0.22963388 0.52573156 0.38023755]
new shoulder:  [[0.        ]
 [0.4285414 ]
 [0.        ]
 [0.04274894]
 [0.        ]
 [0.4285414 ]
 [0.        ]
 [0.04274894]]
alt:  [0.5017172  0.22954907 0.5362266  0.38004786]
new shoulder:  [[0.        ]
 [0.42798242]
 [0.        ]
 [0.04266321]
 [0.        ]
 [0.42798242]
 [0.        ]
 [0.04266321]]
alt:  [0.51247185 0.23005927 0.5563353  0.37853822]
new shoulder:  [[0.        ]
 [0.42764002]
 [0.        ]
 [0.04260371]
 [0.        ]
 [0.42764002]
 [0.        ]
 [0.04260371]]
alt:  [0.5227945  0.23048511 0.58014524 0.37844718]
new shoulder:  [[0.        ]
 [0.42763302]
 [0. 

alt:  [0.5112115  0.18197244 0.5081102  0.4238806 ]
new shoulder:  [[0.        ]
 [0.40807998]
 [0.        ]
 [0.04315475]
 [0.        ]
 [0.40807998]
 [0.        ]
 [0.04315475]]
alt:  [0.52142835 0.1781431  0.54577714 0.43260565]
new shoulder:  [[0.        ]
 [0.4011384 ]
 [0.        ]
 [0.04303579]
 [0.        ]
 [0.4011384 ]
 [0.        ]
 [0.04303579]]
alt:  [0.5343312  0.17315085 0.56701314 0.45057508]
new shoulder:  [[0.        ]
 [0.39833552]
 [0.        ]
 [0.04433664]
 [0.        ]
 [0.39833552]
 [0.        ]
 [0.04433664]]
alt:  [0.56248754 0.18074061 0.60069335 0.45741725]
new shoulder:  [[0.        ]
 [0.40324494]
 [0.        ]
 [0.04717801]
 [0.        ]
 [0.40324494]
 [0.        ]
 [0.04717801]]
alt:  [0.5831132  0.18196684 0.61925876 0.4553346 ]
new shoulder:  [[0.        ]
 [0.40819305]
 [0.        ]
 [0.04927706]
 [0.        ]
 [0.40819305]
 [0.        ]
 [0.04927706]]
alt:  [0.5858393  0.18287998 0.61940455 0.46700537]
new shoulder:  [[0.        ]
 [0.41485167]
 [0. 

alt:  [0.92509395 0.6262678  0.9659337  0.9278145 ]
new shoulder:  [[0.        ]
 [1.1243061 ]
 [0.        ]
 [0.10465463]
 [0.        ]
 [1.1243061 ]
 [0.        ]
 [0.10465463]]
